# Not Used, But Nice Bayesian Model

In [14]:
import math, sys
from konlpy.tag import Twitter
import json

class BayesianFilter:
    def __init__(self):
        self.words = set()
        self.word_dict = {}
        self.category_dict = {}
        
    def split(self, text):
        results = []
        twitter = Twitter()
        malist = twitter.pos(text, norm=True, stem=True)
        for word in malist:
            if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Alpha"]:
                results += word[0]
        return results
    
    def inc_word(self, word, category):
        if not category in self.word_dict:
            self.word_dict[category] = {}
        if not word in self.word_dict[category]:
            self.word_dict[category][word] = 0
        self.word_dict[category][word] += 1
        self.words.add(word)
    
    def inc_category(self, category):
        if not category in self.category_dict:
            self.category_dict[category] = 0
        self.category_dict[category] += 1
        
    def fit(self, text, category):
        for word in text:
            self.inc_word(word, category)
        self.inc_category(category)
        
    def score(self, words, category):
        score = math.log(self.category_prob(category))
        for word in words:
            score += math.log(self.word_prob(word, category))
        return score
    
    def predict(self, text):
        best_category = None
        max_score = -sys.maxsize
        score_list = []
        for category in self.category_dict.keys():
            score = self.score(text, category)
            score_list.append((category, score))
            if score > max_score:
                max_score = score
                best_category = category
        return best_category, score_list
    
    def get_word_count(self, word, category):
        if word in self.word_dict[category]:
            return self.word_dict[category][word]
        else:
            return 0
        
    def category_prob(self, category):
        sum_categories = sum(self.category_dict.values())
        category_v = self.category_dict[category]
        return category_v / sum_categories
    
    def word_prob(self, word, category):
        n = self.get_word_count(word, category) + 1
        d = sum(self.word_dict[category].values()) + len(self.words)
        return n /d

In [16]:
# 데이터 정규화하기

with open('test_data.csv') as file:
    csv_data = []
    for line in file.readlines():
        csv_data.append(line.split(','))

regular_data = [["" for x in range(2)] for y in range(len(csv_data))]
regular_label = [["" for x in range(2)] for y in range(len(csv_data))]
print(len(csv_data))

for i in range(0, len(csv_data)):
    regular_data[i][0] = csv_data[i][0] #mpname
    regular_data[i][0] += csv_data[i][1] #mpstand
    regular_data[i][1] = csv_data[i][1] #mpstand
    
    regular_label[i][0] = csv_data[i][2] #pname
    regular_label[i][0] += csv_data[i][3] #pmake
    regular_label[i][1] = csv_data[i][3] #pmake



10001


In [17]:
train_length = len(csv_data) -12;
test_length = 12;
bf = BayesianFilter()
bf_make = BayesianFilter()

In [18]:
for i in range(1, train_length):
    bf.fit(regular_data[i][0], regular_label[i][0]);
    bf_make.fit(regular_data[i][1], regular_label[i][1]);

In [19]:
for i in range(train_length + 1, train_length + test_length):
    pre, scorelist = bf.predict(regular_data[i][0])
    pre_make, scorelist_make = bf_make.predict(regular_data[i][1])
    posibility = scorelist[0][1] 
    position = 0
    posibility_make = scorelist_make[0][1] 
    position_make = 0
    
    print("입력값 = ", regular_data[i][0])
    print("추측 결과 = ", pre)
    print("실제 결과 = ", regular_label[i][0])
    
    for j in range(1, len(scorelist)):
        if (posibility < scorelist[j][1]):
            posibility = scorelist[j][1]
            position = j
    
    print("입력 메이커 = ", regular_data[i][1])
    print("추측 메이커 = ", pre_make)
    print("실제 메이커 = ", regular_label[i][1])
    for j in range(1, len(scorelist_make)):
        if (posibility_make < scorelist_make[j][1]):
            posibility_make = scorelist_make[j][1]
            position_make = j
    
    print("결과 = ", scorelist[position], scorelist_make[position_make])

    print("\n")
    

입력값 =  패티 냉동 품 햄버거 패티 돼지고기 국산 선진 불고기 버거 세트 햄버거 패티 올리브 치아바타 불고기 소스 햄버거 랩 
추측 결과 =  스모크햄쵸핑목우촌

실제 결과 =  불고기버거빵소스냉장패티냉동선진

입력 메이커 =  돼지고기 국산 선진 불고기 버거 세트 햄버거 패티 올리브 치아바타 불고기 소스 햄버거 랩 
추측 메이커 =  선진

실제 메이커 =  선진

결과 =  ('스모크햄쵸핑목우촌\n', -326.755546165342) ('선진\n', -230.23714498337972)


입력값 =  파인애플 과일 쥬스 과즙 음료 과즙 망고 농축액 파인애플 농축액 천년 풍 밉다 자 연속 망고 파인애플 
추측 결과 =  골드파인애플골든엔골드

실제 결과 =  냉장자연속의파인애플망고천년풍밉다

입력 메이커 =  망고 농축액 파인애플 농축액 천년 풍 밉다 자 연속 망고 파인애플 
추측 메이커 =  천년풍밉다

실제 메이커 =  천년풍밉다

결과 =  ('골드파인애플골든엔골드\n', -284.98850180902974) ('천년풍밉다\n', -174.09845111372786)


입력값 =  토마토 페이스 트 소 포장 토마토 면 사랑 올 드 캘리포니아 토마토 페이스 트 
추측 결과 =  올드캘리포니아토마토페이스트면사랑

실제 결과 =  올드캘리포니아토마토페이스트면사랑

입력 메이커 =  토마토 면 사랑 올 드 캘리포니아 토마토 페이스 트 
추측 메이커 =  면사랑

실제 메이커 =  면사랑

결과 =  ('올드캘리포니아토마토페이스트면사랑\n', -165.866753018644) ('면사랑\n', -122.03787161390359)


입력값 =  탕수육 냉동 품 소스 포함 탕수육 냉동 품 소스 포함 이 온 물엿 쌀엿 안심 찬 바삭 칸 치킨 강정 소스 
추측 결과 =  오쉐프허니머스타드드레싱오뚜기

실제 결과 =  냉장바사칸강정소스안심찬

입력 메이커 =  이 온 물엿 쌀엿 안심 찬 바삭 칸 치킨 강정 소스 
추측 메이커 =  대상

실제 메이커 =  안

In [9]:
data = input()
data_make = input()

pre, scorelist = bf.predict(data)
pre_make, scorelist_make = bf_make.predict(data_make)
posibility = scorelist[0][1] 
position = 0
posibility_make = scorelist_make[0][1] 
position_make = 0
    
print("입력값 = ", data)
print("추측 결과 = ", pre)
    
for j in range(1, len(scorelist)):
    if (posibility < scorelist[j][1]):
        posibility = scorelist[j][1]
        position = j
    
print("입력 메이커 = ", data_make)
print("추측 메이커 = ", pre_make)
for j in range(1, len(scorelist_make)):
    if (posibility_make < scorelist_make[j][1]):
        posibility_make = scorelist_make[j][1]
        position_make = j
    
print("결과 = ", scorelist[position], scorelist_make[position_make])

사과띠
ㅋㅋ
입력값 =  사과띠
추측 결과 =  "사과 식초 ""오뚜기 "

입력 메이커 =  ㅋㅋ
추측 메이커 =  "경주 법주 "

결과 =  ('"사과 식초 ""오뚜기 "\n', -21.618692258518344) ('"경주 법주 "\n', -22.95377027153339)
